# 2. Open API 활용

### 공공데이터포털 사이트 이용
1. 공공데이터포털 사이트 회원가입/로그인
2. 데이터 검색
3. API 사용 요청, 키 발급
4. API 매뉴얼 확인
5. API 연동 개발 및 테스트

In [1]:
import requests
import pandas as pd
import xmltodict
import json

### 보건복지부-코로나19 감염현황 조회 서비스 (XML)

In [2]:
serviceKey = 'kOziccmdb8NkvACScIS0%2BTPmFFyPeQ1UwPNaOgJAVjtBMY3ewnl6TnLnoPCpNm0ZxNDZ%2FuIeWsf%2BPycQV8ie9A%3D%3D'

url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson?serviceKey={}&pageNo=1&numOfRows=10&startCreateDt=20200310&endCreateDt=20200830'.format(serviceKey)
url

'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson?serviceKey=kOziccmdb8NkvACScIS0%2BTPmFFyPeQ1UwPNaOgJAVjtBMY3ewnl6TnLnoPCpNm0ZxNDZ%2FuIeWsf%2BPycQV8ie9A%3D%3D&pageNo=1&numOfRows=10&startCreateDt=20200310&endCreateDt=20200830'

In [3]:
res = requests.get(url)
res

<Response [200]>

In [ ]:
res.text

In [ ]:
temp = xmltodict.parse(res.text)
temp

In [ ]:
result = json.loads(json.dumps(temp) ) # OrderDict -> dict(type:str) -> dict
result

In [ ]:
items = result['response']['body']['items']['item']

for item in items:
    print(item)

In [ ]:
# 일자(STATE_DT), 확진자수(DECIDE_CNT), 사망자수(DEATH_CNT), 치료중환자수(CARE_CNT)
state_dt = []
decide_cnt = []
death_cnt = []
care_cnt = []

for item in items:
    state_dt.append(item['stateDt'])
    decide_cnt.append(item['decideCnt'])
    death_cnt.append(item['deathCnt'])
    care_cnt.append(item['careCnt'])
    
state_dt

In [ ]:
# dataframe 생성
df_data = {
    '기준일': state_dt,
    '확진자수': decide_cnt,
    '사망자수': death_cnt,
    '치료중환자수': care_cnt
}
df_data

In [10]:
df = pd.DataFrame(data=df_data)
df

,기준일,확진자수,사망자수,치료중환자수
0,20200830,19698,323,4313
1,20200829,19399,321,4209
2,20200828,19076,316,3931
3,20200827,18705,313,3584
4,20200826,18264,312,3348
...,...,...,...,...
169,20200314,8086,72,7402
170,20200313,7979,67,7470
171,20200312,7869,66,7407
172,20200311,7755,60,7212


### 한국관광공사 - 관광정보서비스 (JSON)

In [11]:
pageNo = 1
url = 'http://api.visitkorea.or.kr/openapi/service/rest/KorService/areaCode?ServiceKey={}&numOfRows=10&pageNo={}&MobileOS=ETC&MobileApp=AppTest&_type=json'.format(serviceKey, pageNo)
url

'http://api.visitkorea.or.kr/openapi/service/rest/KorService/areaCode?ServiceKey=kOziccmdb8NkvACScIS0%2BTPmFFyPeQ1UwPNaOgJAVjtBMY3ewnl6TnLnoPCpNm0ZxNDZ%2FuIeWsf%2BPycQV8ie9A%3D%3D&numOfRows=10&pageNo=1&MobileOS=ETC&MobileApp=AppTest&_type=json'

In [12]:
res = requests.get(url)
res

<Response [200]>

In [13]:
res.text

'{"response":{"header":{"resultCode":"0000","resultMsg":"OK"},"body":{"items":{"item":[{"code":1,"name":"서울","rnum":1},{"code":2,"name":"인천","rnum":2},{"code":3,"name":"대전","rnum":3},{"code":4,"name":"대구","rnum":4},{"code":5,"name":"광주","rnum":5},{"code":6,"name":"부산","rnum":6},{"code":7,"name":"울산","rnum":7},{"code":8,"name":"세종특별자치시","rnum":8},{"code":31,"name":"경기도","rnum":9},{"code":32,"name":"강원도","rnum":10}]},"numOfRows":10,"pageNo":1,"totalCount":17}}}'

In [14]:
data = res.json()
data

{'response': {'header': {'resultCode': '0000', 'resultMsg': 'OK'},
  'body': {'items': {'item': [{'code': 1, 'name': '서울', 'rnum': 1},
     {'code': 2, 'name': '인천', 'rnum': 2},
     {'code': 3, 'name': '대전', 'rnum': 3},
     {'code': 4, 'name': '대구', 'rnum': 4},
     {'code': 5, 'name': '광주', 'rnum': 5},
     {'code': 6, 'name': '부산', 'rnum': 6},
     {'code': 7, 'name': '울산', 'rnum': 7},
     {'code': 8, 'name': '세종특별자치시', 'rnum': 8},
     {'code': 31, 'name': '경기도', 'rnum': 9},
     {'code': 32, 'name': '강원도', 'rnum': 10}]},
   'numOfRows': 10,
   'pageNo': 1,
   'totalCount': 17}}}

In [15]:
numOfRows = data['response']['body']['numOfRows']
totalCount = data['response']['body']['totalCount']

numOfRows, totalCount

(10, 17)

In [16]:
queryCount = totalCount // numOfRows
if totalCount % numOfRows > 0:
    queryCount += 1
queryCount

2

In [ ]:
code = []
name = []
rnum = []

for i in range(queryCount):  # 0~1
    pageNo = i + 1  #1~2
    'http://api.visitkorea.or.kr/openapi/service/rest/KorService/areaCode?ServiceKey={}&numOfRows=10&pageNo={}&MobileOS=ETC&MobileApp=AppTest&_type=json'.format(serviceKey, pageNo)
    res = requests.get(url)
    data = res.json()
    
    itmes = data['response']['body']['items']['item']
    
    for item in items:
        code.append(item['code'])
        name.append(item['name'])
        rnum.append(item['rnum'])
        
name

In [ ]:
df_data = {
    'code': code,
    'name': name,
    'rnum': rnum
}

df = pd.DataFrame(data=df_data)
df